In [53]:
import pandas as pd, numpy as np, json, math
import datetime as dt

In [54]:
from obs.models import Observation

## This is for observations that are keyed to existing sites
see Geometry notebook for creating new sites from lat, long columns

In [69]:
df = pd.read_csv('infiltration_odk.csv')
#watch out for missing or blank fields, do cleanup as needed

In [56]:
df = df.rename(columns={'site_description':'description','date':'olddate'})

In [57]:
#add foreign key fields for atlasbiowork Observation model
df['observer_id'] = 1 # this is my observer_id
df.rename(columns={'pk':'site_id'}, inplace=True)
df['type_id'] = 19 #for infiltration

In [68]:
#Collect timings columns in format 'mm:ss, mm:ss, mm:ss'
#convert decimal minutes to mm:ss
def pad(n):
    if n > 0:
        return str(math.floor(n)) + ':' + str.zfill(str(math.floor((n % 1)*60)),2)
    else:
        return ''

timing_cols=['ring_1:inf_time1','ring_1:inf_time2','ring_1:inf_time3','ring_1:inf_time4']   
for i, row in df[timing_cols].iterrows():
    l = list(row)
    m = list(pad(value) for value in l if not math.isnan(value))
    df.loc[i,'timings_1'] = ', '.join(m)


In [59]:
#we need YYYY-MM-DD format as string for the JSON field
df['date'] = pd.to_datetime(df['olddate']).dt.strftime('%Y-%m-%d')


this next part is tricky. Must use JSON functions, not to_dict().

In [60]:
infil_values = ['timings_1','description','date']#columns for JSON values field for this datatype.
s = df[infil_values].to_json(orient='records')
t = pd.Series(json.loads(s))
v = pd.DataFrame(t, columns=['values'])
#now we have a JSON values field that we can add to df

In [61]:
#reduce unneeded columns
df= df[['site_id',
       'observer_id', 'type_id']]

In [62]:
#merge in the values columns. The indexes should correspond since they are from the same df
df = df.merge(v, left_index=True, right_index=True)

In [63]:
#save it for good measure
df.to_csv('infiltrationNEW.csv', index=False)

<font color= red>WARNING: db operation ahead</font>
don't use bulk_create. Values field has to be a dict, not a str

In [52]:
#now this seems to work!!!
for row in df.itertuples():
    p = Observation(
        site_id=row[1],
        observer_id=row[2],
        type_id=row[3],
        values=row[4])
    #p.save()  #UNCOMMENT AT YOUR PERIL!!!! 
    
    